In [23]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
%cd /content/gdrive/MyDrive/Dementia/
!ls

/content/gdrive/MyDrive/Dementia
'Alzheimer_s Dataset'   Main.ipynb
'Copy of Main.ipynb'    TensorFlowLite2_Object_Detction_Model.ipynb


In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

In [26]:
# Allow TensorFlow to use the full GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [27]:
# Set the path to your dataset folder
dataset_path = 'Alzheimer_s Dataset/'


In [28]:
# Parameters
image_size = (176, 176)  # Resize images to this size
batch_size = 64  # Increase the batch size to use more GPU memory
epochs = 500  # Increase the number of epochs for potentially better accuracy


In [29]:
# Load the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(dataset_path, 'train'),
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int'
)

Found 5121 files belonging to 4 classes.


In [30]:
# Load the validation dataset
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(dataset_path, 'valid'),
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int'
)

Found 1279 files belonging to 4 classes.


In [31]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
])

In [32]:

# Normalize the pixel values to be between 0 and 1
normalization_layer = layers.Rescaling(1./255)

# Apply data augmentation to the training dataset
augmented_train_ds = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
normalized_train_ds = augmented_train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetch to improve performance
AUTOTUNE = tf.data.AUTOTUNE
normalized_train_ds = normalized_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalized_val_ds = normalized_val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [33]:
# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(176, 176, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_dataset.class_names), activation='softmax')  # Adjust the output layer based on the number of classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [34]:
# Print the model summary
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 174, 174, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 87, 87, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 85, 85, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 42, 42, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 40, 40, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 20, 20, 128)      

In [35]:
# Define callbacks for early stopping and saving the best model
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True)
]



In [36]:
# Define early stopping callback with increased patience
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=20,          # Increase patience to allow training to continue for more epochs
    restore_best_weights=True  # Restore weights of the best performing model
)



In [37]:
# Train the model with early stopping callback
history = model.fit(
    normalized_train_ds,
    validation_data=normalized_val_ds,
    epochs=500,
    callbacks=[early_stopping]  # Pass the early stopping callback
)


Epoch 1/500
81/81 [==============================] - 68s 406ms/step - loss: 1.1228 - accuracy: 0.4888 - val_loss: 1.0066 - val_accuracy: 0.5004
Epoch 2/500
81/81 [==============================] - 5s 67ms/step - loss: 1.0050 - accuracy: 0.5216 - val_loss: 1.0999 - val_accuracy: 0.4246
Epoch 3/500
81/81 [==============================] - 5s 66ms/step - loss: 0.9793 - accuracy: 0.5255 - val_loss: 0.9632 - val_accuracy: 0.5364
Epoch 4/500
81/81 [==============================] - 5s 65ms/step - loss: 0.9134 - accuracy: 0.5532 - val_loss: 0.9608 - val_accuracy: 0.5575
Epoch 5/500
81/81 [==============================] - 5s 67ms/step - loss: 0.8995 - accuracy: 0.5641 - val_loss: 0.9302 - val_accuracy: 0.5559
Epoch 6/500
81/81 [==============================] - 5s 67ms/step - loss: 0.8657 - accuracy: 0.5903 - val_loss: 0.9657 - val_accuracy: 0.5473
Epoch 7/500
81/81 [==============================] - 5s 66ms/step - loss: 0.8375 - accuracy: 0.6038 - val_loss: 0.9409 - val_accuracy: 0.5536
Epoc

In [38]:
# Save the trained model in TensorFlow SavedModel format
model.save('image_classification_model')

# Convert and save the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('image_classification_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(normalized_val_ds)
print(f"Validation accuracy: {val_accuracy * 100:.2f}%")

20/20 [==============================] - 0s 24ms/step - loss: 0.9302 - accuracy: 0.5559
Validation accuracy: 55.59%
